In [ ]:
# Exp Data Analysis
# https://towardsdatascience.com/a-machine-learning-project-predicting-used-car-prices-efbc4d2a4998#eced

In [2]:
%matplotlib inline

import quandl
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from dateutil.parser import parse
from sklearn import metrics

In [3]:
dfX = pd.read_csv("portfolio.csv", sep='\t')
dfX.set_index('Date')

,AAPL,AMZN,EXC,NDAQ,RGLD,Portfolio Val
Date,,,,,,
2015-07-01,2500.000000,2500.000000,1000.000000,3000.000000,1000.000000,10000.000000
2015-07-02,2496.840442,2501.829031,1010.085093,3025.867378,1020.802621,10055.424566
2015-07-06,2488.151659,2492.283774,1006.618342,3003.079450,1038.902539,10029.035764
2015-07-07,2482.030016,2496.170466,1032.776552,3008.006569,1009.500410,10028.484013
2015-07-08,2420.418641,2456.046092,1027.418847,2947.649353,1011.711712,9863.244645
...,...,...,...,...,...,...
2017-12-22,3608.659917,6678.021903,1340.219995,4888.929875,1467.426587,17983.258276
2017-12-26,3517.108291,6726.033974,1333.348834,4891.499952,1485.905041,17953.896092
2017-12-27,3517.726883,6757.470450,1339.532879,4904.992859,1470.308547,17990.031618


In [4]:
dfX = dfX[['Portfolio Val']]
dfX['Portfolio Val'] = dfX['Portfolio Val'].astype(float)
dfX

,Portfolio Val
0,10000.000000
1,10055.424566
2,10029.035764
3,10028.484013
4,9863.244645
...,...
624,17983.258276
625,17953.896092
626,17990.031618
627,17974.392222


In [5]:
dfX['Rolling_mean_PV'] = dfX['Portfolio Val'].rolling(5).mean()
dfX['Rolling_std_PV'] = dfX['Portfolio Val'].rolling(5).std()
dfX['upper_band_PV'] = dfX['Rolling_mean_PV']+ (2*dfX['Rolling_std_PV'])
dfX['lower_band_PV'] = dfX['Rolling_mean_PV']- (2*dfX['Rolling_std_PV'])
dfX.fillna(dfX.mean())

,Portfolio Val,Rolling_mean_PV,Rolling_std_PV,upper_band_PV,lower_band_PV
0,10000.000000,13369.735506,104.566367,13578.868239,13160.602773
1,10055.424566,13369.735506,104.566367,13578.868239,13160.602773
2,10029.035764,13369.735506,104.566367,13578.868239,13160.602773
3,10028.484013,13369.735506,104.566367,13578.868239,13160.602773
4,9863.244645,9995.237797,76.346138,10147.930074,9842.545520
...,...,...,...,...,...
624,17983.258276,18034.986713,109.541511,18254.069735,17815.903691
625,17953.896092,17983.406597,50.021882,18083.450361,17883.362832
626,17990.031618,17967.594727,19.109936,18005.814600,17929.374854
627,17974.392222,17973.523356,14.224080,18001.971516,17945.075196


In [6]:
data = pd.read_csv("S&P500.csv")
data['Date']  # as a Series
date_array = data['Date'].values  # as a numpy array
d_list = []
for date in date_array:
    dt = parse(date)
    d_item = (str(dt.date()))    
    d_list.append(d_item)
dfDate = pd.DataFrame(d_list,columns=['Date'])
dfDate.set_index('Date')
dfSP500 = pd.read_csv("S&P500.csv")
dfSP500.set_index('Date')
dfSP500 = dfSP500.rename(columns={'Price':'SP500', 'Date':'ODate'})

for i, col in enumerate(dfSP500.columns):
    dfSP500.iloc[:, i] = dfSP500.iloc[:, i].str.replace(',', '')
    
dfSP500 = dfSP500.join(dfDate)
dfSP500 = dfSP500.dropna()
dfSP500 = dfSP500[['SP500']]
dfSP500['SP500'] = dfSP500['SP500'].astype(float)
dfSP500 = dfSP500[::-1]
dfSP500.reset_index(inplace=True, drop=True)

In [7]:
dfX = dfX.join(dfSP500)
dfX.dropna()

,Portfolio Val,Rolling_mean_PV,Rolling_std_PV,upper_band_PV,lower_band_PV,SP500
4,9863.244645,9995.237797,76.346138,10147.930074,9842.545520,2046.68
5,9924.323042,9980.102406,82.425353,10144.953111,9815.251701,2051.31
6,10107.138356,9990.445164,96.314065,10183.073294,9797.817034,2076.62
7,10244.621207,10033.562252,150.770029,10335.102310,9732.022195,2099.60
8,10319.947293,10091.854909,197.437864,10486.730637,9696.979180,2108.95
...,...,...,...,...,...,...
624,17983.258276,18034.986713,109.541511,18254.069735,17815.903691,2679.25
625,17953.896092,17983.406597,50.021882,18083.450361,17883.362832,2684.57
626,17990.031618,17967.594727,19.109936,18005.814600,17929.374854,2683.34
627,17974.392222,17973.523356,14.224080,18001.971516,17945.075196,2680.50


In [8]:
dfX.describe()

,Portfolio Val,Rolling_mean_PV,Rolling_std_PV,upper_band_PV,lower_band_PV,SP500
count,629.000000,625.000000,625.000000,625.000000,625.000000,629.000000
mean,13373.595923,13369.735506,104.566367,13578.868239,13160.602773,2223.049141
std,2352.693243,2335.808491,65.500885,2324.243397,2354.616353,207.102255
min,9500.286457,9839.985634,5.041378,10144.953111,9048.230215,1829.080000
25%,11281.620234,11271.397308,61.324932,11466.753995,11091.556441,2071.180000
50%,13302.268737,13303.927717,90.126445,13519.850888,13092.697880,2168.270000
75%,15365.576956,15363.311964,135.399861,15604.631598,15041.415126,2396.920000
max,18309.025416,18207.281931,528.000548,18504.266930,18027.006482,2690.160000


In [9]:
dfX[4:].to_csv('XY_var.csv', sep='\t', encoding='utf-8')

In [10]:
y = pd.read_csv("XY_var.csv", sep='\t')

In [11]:
y = y['Portfolio Val'].to_numpy()

In [13]:
y.shape

(625,)

In [14]:
x = pd.read_csv("XY_var.csv", sep='\t')

In [15]:
x = x[['SP500', 'Rolling_mean_PV', 'Rolling_std_PV', 'upper_band_PV', 'lower_band_PV']]

In [16]:
x = x.to_numpy()

In [17]:
x.shape

(625, 5)

In [18]:
x

array([[2.04668000e+03, 9.99523780e+03, 7.63461385e+01, 1.01479301e+04,
        9.84254552e+03],
       [2.05131000e+03, 9.98010241e+03, 8.24253526e+01, 1.01449531e+04,
        9.81525170e+03],
       [2.07662000e+03, 9.99044516e+03, 9.63140651e+01, 1.01830733e+04,
        9.79781703e+03],
       ...,
       [2.68334000e+03, 1.79675947e+04, 1.91099364e+01, 1.80058146e+04,
        1.79293749e+04],
       [2.68050000e+03, 1.79735234e+04, 1.42240800e+01, 1.80019715e+04,
        1.79450752e+04],
       [2.68262000e+03, 1.79516035e+04, 5.49079611e+01, 1.80614194e+04,
        1.78417876e+04]])

In [19]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20)

# Decision Tree Model

In [20]:
from sklearn.tree import DecisionTreeRegressor
regressor = DecisionTreeRegressor()
regressor.fit(x_train, y_train)

DecisionTreeRegressor()

In [21]:
y_pred = regressor.predict(x_test)

In [22]:
df=pd.DataFrame({'Actual':y_test, 'Predicted':y_pred})
df

,Actual,Predicted
0,11403.991618,11290.351085
1,10244.621207,9863.244645
2,14501.440919,14550.819005
3,17868.954022,17896.836308
4,15584.191935,15569.958674
...,...,...
120,11334.528769,11291.135709
121,16411.187764,16192.932029
122,16278.007310,16683.294828
123,13502.716616,13519.342468


In [23]:
from sklearn import metrics
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

Mean Absolute Error: 108.85384332946005
Mean Squared Error: 22355.862981203958
Root Mean Squared Error: 149.51877133391633


# Random Forest Regressor

In [35]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.50)

In [36]:
model = RandomForestRegressor(random_state=0).fit(x_train, y_train)
model.score(x_train, y_train), model.score(x_test, y_test)

(0.9994653099074157, 0.9970135727503526)

In [37]:
y_pred=model.predict(x_test)

In [38]:
df=pd.DataFrame({'Actual':y_test, 'Predicted':y_pred})
df

,Actual,Predicted
0,12395.350598,12327.323483
1,11691.318603,11687.944535
2,10088.025546,10093.610936
3,10138.808953,10264.475647
4,10457.885789,10473.748756
...,...,...
308,10405.984236,10415.904833
309,10474.599498,10366.438294
310,13072.549920,12845.579707
311,10138.833666,10248.652684


In [39]:
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

Mean Absolute Error: 91.169219716782
Mean Squared Error: 16499.8297102867
Root Mean Squared Error: 128.451662933131


In [40]:
bagging = RandomForestRegressor(max_features=None)# or we can pass max_features = len(X_train.columns) = 13
bagging.fit(x_train,y_train)
print('Training score ',bagging.score(x_train,y_train))
print('Test Error ',bagging.score(x_test,y_test))

Training score  0.9994590267972097
Test Error  0.9971082522796304


In [41]:
rf = RandomForestRegressor(max_features = 1) # same as book
rf.fit(x_train,y_train)
print('Training score ',rf.score(x_train,y_train))
print('Test Error ',rf.score(x_test,y_test))

Training score  0.9994171021178991
Test Error  0.997148517100727


In [44]:
features = pd.get_dummies(dfX)

In [45]:
# Import tools needed for visualization
from sklearn.tree import export_graphviz
import pydot

# Remove the labels from the features
# axis 1 refers to the columns
#features= features.drop('Portfolio Val', axis = 1)

# Saving feature names for later use
feature_list = list(features.columns)

# Pull out one tree from the forest
tree = rf.estimators_[5]

# Export the image to a dot file
export_graphviz(tree, out_file = 'tree.dot', feature_names = feature_list, rounded = True, precision = 1)

# Use dot file to create a graph
(graph, ) = pydot.graph_from_dot_file('tree.dot')

# Write graph to a png file
graph.write_png('tree.png'); 

FileNotFoundError: [Errno 2] "dot" not found in path.

In [31]:
feat_importances = pd.Series(rf.feature_importances_, index=dfX.head()[1:])
feat_importances.nlargest(25).plot(kind='barh',figsize=(10,10))

ValueError: Index data must be 1-dimensional